In [4]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import os
from PIL import Image
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import torch.nn.functional as F
import random

In [5]:
import warnings
warnings.filterwarnings("ignore")

# pre-defined
category_dict = {'c1':'Men Tshirts', 'c2':'Sarees', 'c3': 'Kurtis', 'c4': 'Women Tshirts', 'c5': 'Women Tops & Tunics'}
semi_classes_dict = {'c1':[4, 2, 2, 3, 2], 'c2':[4, 6, 3, 8, 4, 3, 4, 5, 9, 2], 'c3': [13, 2, 2, 2, 2, 2, 2, 3, 2], 'c4': [7, 3, 3, 3, 6, 3, 2, 2], 'c5': [12, 4, 2, 7, 2, 3, 6, 4, 4, 6]}

In [6]:
# set paths as per the set-up and Hyperparameters
input_path = "/kaggle/input/visual-taxonomy"
working_path = "/kaggle/working"
test_c_name = "c3"
NUM_EPOCH = 40
LEARNING_RATE = 0.001
NUM_OF_SEMI_CLASSES_OF_COLUMNS = semi_classes_dict[test_c_name]

In [7]:
df_train = pd.read_csv(f'{input_path}/train.csv')
df_test = pd.read_csv(f'{input_path}/test.csv')

In [8]:
def replace_na_values_with_highest_class(df_obj):

    # Select columns from 'attr_1' to the end of the dataframe
    attr_columns = df_obj.loc[:, 'attr_1':]
    
    # Replace NaN values with the most frequent class in each column
    for col in attr_columns.columns:
        # Find the most frequent value in the column
        highest_class = attr_columns[col].value_counts().idxmax()
        # Replace NaN values in the column with the most frequent value
        df_obj[col].fillna(highest_class, inplace=True)

    return df_obj

In [9]:
def do_preprocessing(test_c_name, test_category):
    df_sub = df_train[df_train['Category'] == test_category]
    print(df_sub.info())
    df_sub.dropna(axis=1, how='all', inplace=True)
    df_sub = replace_na_values_with_highest_class(df_obj = df_sub)
    print(df_sub.info())
    return df_sub

In [10]:
if test_c_name == "c1":
    df_c1 = do_preprocessing(test_c_name, category_dict[test_c_name])
elif test_c_name == "c2":
    df_c2 = do_preprocessing(test_c_name, category_dict[test_c_name])
elif test_c_name == "c3":
    df_c3 = do_preprocessing(test_c_name, category_dict[test_c_name])
elif test_c_name == "c4":
    df_c4 = do_preprocessing(test_c_name, category_dict[test_c_name])
elif test_c_name == "c5":
    df_c5 = do_preprocessing(test_c_name, category_dict[test_c_name])

<class 'pandas.core.frame.DataFrame'>
Index: 6822 entries, 25613 to 32434
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        6822 non-null   int64 
 1   Category  6822 non-null   object
 2   len       6822 non-null   int64 
 3   attr_1    6629 non-null   object
 4   attr_2    3231 non-null   object
 5   attr_3    3400 non-null   object
 6   attr_4    6431 non-null   object
 7   attr_5    3266 non-null   object
 8   attr_6    3848 non-null   object
 9   attr_7    3843 non-null   object
 10  attr_8    6702 non-null   object
 11  attr_9    6691 non-null   object
 12  attr_10   0 non-null      object
dtypes: int64(2), object(11)
memory usage: 746.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 6822 entries, 25613 to 32434
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        6822 non-null   int64 
 1   Category  6822 non-null   object
 2   l

In [11]:
class LabelEncoderDict:
    def __init__(self):
        self.encoders = {}
    def fit(self, df, columns):
        """Fit label encoders for each column"""
        for col in columns:
            le = LabelEncoder()
            # Include NaN as a unique label by appending it to valid labels
            valid_labels = df[col].dropna().unique().tolist()
            # valid_labels.append('NaN')  # Assign a label for NaN
            le.fit(valid_labels)
            self.encoders[col] = le
    def transform(self, df, columns):
        """Transform labels using fitted encoders"""
        encoded = np.zeros((len(df), len(columns)))
        for i, col in enumerate(columns):
            series = df[col].copy()
            # Replace NaNs with the string 'NaN' so they can be encoded
            # series = series.fillna('NaN')
            encoded[:, i] = self.encoders[col].transform(series)
        return encoded
    def get_num_classes(self, column):
        """Get number of classes for a specific column"""
        return len(self.encoders[column].classes_)
        
class MultiLabelImageDataset(Dataset):
    def __init__(self, df, image_dir, transform_basic=None, transform_augmented=None, attr_columns=10,do_transform=True):
        self.df = df
        self.image_dir = image_dir
        self.transform_basic = transform_basic  # Basic transform without augmentation
        self.transform_augmented = transform_augmented  # Augmented transform with augmentation
        self.attr_columns = attr_columns
        self.do_transform = do_transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        # Get image path
        img_name = str(self.df.iloc[idx]['id']).zfill(6)
        img_path = os.path.join(self.image_dir, f"{img_name}.jpg")
        # Load image
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            image = Image.new('RGB', (512,512))
        # Apply random probability to choose between augmentation or notF
        if (random.random() > 0.5) and self.do_transform:
            if self.transform_basic:
                image = self.transform_basic(image)
        else:
            if self.transform_augmented:
                image = self.transform_augmented(image)
        # Ensure labels are integers and convert to tensor
        labels = torch.tensor(self.df.iloc[idx][self.attr_columns].astype(int).values, dtype=torch.long)
        return image, labels
        
def prepare_data(df, image_dir, batch_size=32, test_size=0.2, num_attr_columns=10):
    """
    Prepare data loaders and label encoders
    """
    # Define attribute columns
    attr_columns = [f'attr_{i}' for i in range(1, num_attr_columns+1)]
    # Create and fit label encoders
    label_encoders = LabelEncoderDict()
    label_encoders.fit(df, attr_columns)
    # Transform labels
    encoded_labels = label_encoders.transform(df, attr_columns)
    df_encoded = df.copy()
    for i, col in enumerate(attr_columns):
        df_encoded[col] = encoded_labels[:, i]
    # Split data
    train_df, val_df = train_test_split(df_encoded, test_size=test_size, random_state=42)
    # Define transforms
    transform = transforms.Compose([
        transforms.Resize((512,512)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])
    transform_augmented = transforms.Compose([
    transforms.Resize((512,512)), 
    transforms.RandomRotation(degrees=15),  # Rotate by up to 15 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Adjust color properties
    transforms.RandomResizedCrop(size=(512, 512), scale=(0.8, 1.0)),  # Randomly crop and resize
    transforms.RandomPerspective(distortion_scale=0.1, p=0.5),  # Apply perspective distortion
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=10),  # Affine transformations
    transforms.GaussianBlur(kernel_size=5, sigma=(0.1, 2.0)),  # Apply random Gaussian blur
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize based on ImageNet statistics
    ])
    # Create datasets
    train_dataset = MultiLabelImageDataset(
        train_df,
        image_dir,
        transform_basic=transform,
        transform_augmented = transform_augmented,
        attr_columns=attr_columns
    )
    val_dataset = MultiLabelImageDataset(
        val_df,
        image_dir,
        transform_basic=transform,
        transform_augmented = transform_augmented,
        attr_columns=attr_columns,
        do_transform = False
    )
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    # Get number of classes for each attribute
    num_classes_per_attr = [label_encoders.get_num_classes(col) for col in attr_columns]
    return train_loader, val_loader, label_encoders, num_classes_per_attr

class MultiLabelCELoss(nn.Module):
    def __init__(self):
        super(MultiLabelCELoss, self).__init__()
        self.criterion = nn.CrossEntropyLoss(label_smoothing=0.05)
    def forward(self, outputs, targets):
        # outputs is a list of predictions for each label
        # targets is a tensor of shape (batch_size, num_labels)
        loss = 0
        for i, output in enumerate(outputs):
            loss += self.criterion(output, targets[:, i])
        return loss / len(outputs)

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from tqdm import tqdm 

from transformers import ConvNextModel
class MultiLabelClassifier(nn.Module):
    def __init__(self, num_classes_per_attr):
        super(MultiLabelClassifier, self).__init__()
        # Use ConvNeXt-Base with unfrozen backbone
        self.backbone = ConvNextModel.from_pretrained("facebook/convnext-base-384-22k-1k")
        backbone_features = self.backbone.config.hidden_sizes[-1]  # 1024 for base model
        # Modified feature processing without fixed dimensions
        self.feature_processor = nn.Sequential(
            nn.Conv2d(backbone_features, 1024, kernel_size=1),
            nn.GELU(),
            nn.Dropout(0.1)
        )
        # Complex classifier heads for each attribute
        self.classifier_heads = nn.ModuleList()
        for num_classes in num_classes_per_attr:
            classifier_head = nn.Sequential(
                # First branch - Spatial attention
                nn.Sequential(
                    nn.Conv2d(1024, 512, kernel_size=3, padding=1, groups=32),
                    nn.GELU(),
                    nn.Conv2d(512, 512, kernel_size=3, padding=1, groups=32),
                    nn.GELU(),
                ),
                # Second branch - Channel attention (SE-like module)
                nn.Sequential(
                    nn.AdaptiveAvgPool2d(1),
                    nn.Flatten(),
                    nn.Linear(512, 128),
                    nn.GELU(),
                    nn.Linear(128, 512),
                    nn.Sigmoid(),
                ),
                # Combine branches and final classification
                nn.Sequential(
                    nn.AdaptiveAvgPool2d(1),
                    nn.Flatten(),
                    nn.Linear(512, 1024),
                    nn.LayerNorm(1024),
                    nn.GELU(),
                    nn.Dropout(0.2),
                    nn.Linear(1024, 512),
                    nn.LayerNorm(512),
                    nn.Sigmoid(),
                    nn.Dropout(0.1),
                    nn.Linear(512, num_classes)
                )
            )
            self.classifier_heads.append(classifier_head)
  
    def forward(self, x):
        # Extract features from ConvNeXt backbone
        features = self.backbone(x).last_hidden_state
        # Process features
        processed_features = self.feature_processor(features)
        outputs = []
        for classifier_head in self.classifier_heads:
            # Spatial attention branch
            spatial_features = classifier_head[0](processed_features)
            # Channel attention branch
            channel_attention = classifier_head[1](spatial_features)
            channel_attention = channel_attention.view(-1, 512, 1, 1)
            # Apply channel attention and get final output
            attended_features = spatial_features * channel_attention
            output = classifier_head[2](attended_features)
            outputs.append(output)
        return outputs
        
class MultiLabelCELoss(nn.Module):
    def __init__(self):
        super(MultiLabelCELoss, self).__init__()
        self.criterion = nn.CrossEntropyLoss()
    def forward(self, outputs, targets):
        loss = 0
        for i, output in enumerate(outputs):
            loss += self.criterion(output, targets[:, i])
        return loss / len(outputs)


from tqdm import tqdm  # Import tqdm for progress tracking
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch
from tqdm import tqdm
import torch.nn as nn


def train_model(model, train_loader, val_loader, num_epochs, num_classes_per_attr, model_type):
    model = torch.nn.DataParallel(model)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    criterion = MultiLabelCELoss()  # Assuming you have this loss function
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-2)
    # Reduce learning rate on plateau
    lr_scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, verbose=True)
    # Early stopping params
    early_stopping_patience = 4
    early_stopping_counter = 0
    best_val_avg_acc = 0.0
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        correct_predictions = [0] * len(num_classes_per_attr)
        total_predictions = 0
        overall_correct = 0
        with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Training]", unit="batch") as t:
            for images, labels in t:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()
                train_loss += loss.item()
                t.set_postfix(loss=loss.item())
                all_labels_match = torch.ones(labels.size(0), dtype=torch.bool, device=device)
                for i, output in enumerate(outputs):
                    _, predicted = torch.max(output, 1)
                    correct_predictions[i] += (predicted == labels[:, i]).sum().item()
                    all_labels_match &= (predicted == labels[:, i])
                overall_correct += all_labels_match.sum().item()
                total_predictions += labels.size(0)
        # Validation phase
        model.eval()
        val_loss = 0
        val_correct_predictions = [0] * len(num_classes_per_attr)
        val_total_predictions = 0
        val_overall_correct = 0
        with tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Validation]", unit="batch") as v:
            with torch.no_grad():
                for images, labels in v:
                    images, labels = images.to(device), labels.to(device)
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    val_loss += loss.item()
                    v.set_postfix(loss=loss.item())
                    all_labels_match_val = torch.ones(labels.size(0), dtype=torch.bool, device=device)
                    for i, output in enumerate(outputs):
                        _, predicted = torch.max(output, 1)
                        val_correct_predictions[i] += (predicted == labels[:, i]).sum().item()
                        all_labels_match_val &= (predicted == labels[:, i])
                    val_overall_correct += all_labels_match_val.sum().item()
                    val_total_predictions += labels.size(0)
        # Print results
        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'Training Loss: {train_loss/len(train_loader):.4f}')
        print(f'Validation Loss: {val_loss/len(val_loader):.4f}')
        avg_acc = 0
        for i in range(len(num_classes_per_attr)):
            train_acc = 100 * correct_predictions[i] / total_predictions
            val_acc = 100 * val_correct_predictions[i] / val_total_predictions
            avg_acc += val_acc
            print(f'Attribute {i+1} - Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%')
        avg_acc = avg_acc/(len(num_classes_per_attr))
        overall_train_acc = 100 * overall_correct / total_predictions
        overall_val_acc = 100 * val_overall_correct / val_total_predictions
        print(f'Overall Train Accuracy: {overall_train_acc:.2f}%')
        print(f'Overall Validation Accuracy: {overall_val_acc:.2f}%')
        # Early stopping logic based on validation overall accuracy
        if avg_acc >= best_val_avg_acc:
            best_val_avg_acc = avg_acc
            torch.save(model.module.state_dict(), f'best_model_{model_type}.pth')  # Save the best model
            early_stopping_counter = 0  # Reset early stopping counter
        else:
            early_stopping_counter += 1
        # ReduceLROnPlateau 
        lr_scheduler.step(overall_val_acc)
        # Check early stopping condition
        if early_stopping_counter >= early_stopping_patience:
            print("Early stopping triggered")
    torch.save(model.module.state_dict(), f'best_model_end_{model_type}.pth')

In [13]:
import pickle
import gc

def main(test_c_name):
    # Set image directory
    image_dir = f'{input_path}/train_images'
    # Initialize device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # Define a function to train each model sequentially
    def train_single_model(data, num_attr_columns, model_type):
        # Prepare data
        print(f"Preparing data for {model_type}")
        train_loader, val_loader, label_encoders, num_classes_per_attr = prepare_data(data, image_dir, batch_size=16, num_attr_columns=num_attr_columns)
        world_size = torch.cuda.device_count()
        # Initialize the model
        print(f"Initializing model {model_type}")
        model = MultiLabelClassifier(num_classes_per_attr).to(device)
        # Define loss and optimizer
        criterion = MultiLabelCELoss()
        optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
        # Train the model
        print(f"Training model {model_type}")
        train_model(model, train_loader, val_loader, num_epochs=NUM_EPOCH, num_classes_per_attr=num_classes_per_attr, model_type=model_type)
        # Save label encoders
        with open(f'label_encoders_{model_type}.pkl', 'wb') as f:
            pickle.dump(label_encoders, f)
        print("---------------------------------------------")
        print(f"number of classes for {model_type} is {num_classes_per_attr}")
        print("---------------------------------------------")
        # Free up memory
        del model, train_loader, val_loader, label_encoders, num_classes_per_attr
        torch.cuda.empty_cache()
        gc.collect()


    if test_c_name == "c1":
        train_single_model(df_c1, num_attr_columns=5, model_type=test_c_name)
    elif test_c_name == "c2":
        train_single_model(df_c2, num_attr_columns=10, model_type=test_c_name)
    elif test_c_name == "c3":
        train_single_model(df_c3, num_attr_columns=9, model_type=test_c_name)
    elif test_c_name == "c4":
        train_single_model(df_c4, num_attr_columns=8, model_type=test_c_name)
    elif test_c_name == "c5":
        train_single_model(df_c5, num_attr_columns=10, model_type=test_c_name)
    else:
        print("Please do check the name of category, something is wrong.")

In [ ]:
main(test_c_name)

Preparing data for c3
Initializing model c3


config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/354M [00:00<?, ?B/s]

Training model c3


Epoch 1/40 [Training]:   0%|          | 1/342 [00:04<23:51,  4.20s/batch, loss=1.01]

In [ ]:
test_df_semi = df_test[df_test['Category'] == category_dict[test_c_name]]
test_df_semi

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import pickle
from tqdm import tqdm
import time

def load_model(model_path, num_classes_per_attr, device):
    # Initialize the model architecture and load the saved weights
    model = MultiLabelClassifier(num_classes_per_attr)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = torch.nn.DataParallel(model)
    model.to(device)
    model.eval()
    return model
    
def load_label_encoders(encoder_path):
    with open(encoder_path, 'rb') as f:
        encoders = pickle.load(f)
    return encoders

def preprocess_image(image_path, image_size=(512,512)):
    # Define image transformations (same as used during training)
    transform = transforms.Compose([
        transforms.Resize((512,512)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    # Open image and apply transformations
    image = Image.open(image_path).convert('RGB')
    image = transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    return image
    
def inference(image, model, label_encoders):
    # Perform inference
    with torch.no_grad():
        outputs = model(image)
    # Decode predictions
    predicted_labels = []
    for i, output in enumerate(outputs):
        _, predicted = torch.max(output, 1)
        # Attribute name should match the encoder dictionary keys like 'attr_1', 'attr_2', etc.
        attr_name = f'attr_{i + 1}'
        if attr_name in label_encoders.encoders:
            decoded_label = label_encoders.encoders[attr_name].inverse_transform([predicted.item()])[0]
            predicted_labels.append(decoded_label)
        else:
            raise KeyError(f"Encoder for {attr_name} not found in the loaded label encoders.")
    return predicted_labels
    
# Load the model and encoders once
model_path = f"best_model_{test_c_name}.pth"
encoder_path = f"{working_path}/label_encoders_{test_c_name}.pkl"
num_classes_per_attr = NUM_OF_SEMI_CLASSES_OF_COLUMNS
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = load_model(model_path, num_classes_per_attr, device)
label_encoders = load_label_encoders(encoder_path)
image_dir = f"{input_path}/test_images"
preds = {f'attr_{i}': [] for i in range(1, 10)}
t1 = time.time()
for val in tqdm(test_df_semi['id'], desc='Processing Images', total=len(test_df_semi)):
    image_path = f"{image_dir}/{str(val).zfill(6)}.jpg"
    image = preprocess_image(image_path).to(device)  # Preprocess and send image to device
    predictions = inference(image, model, label_encoders)  # Use the already loaded model and encoders
    for i in range(1, 10):
        preds[f'attr_{i}'].append(predictions[i-1])
print(f'Time taken to process images is {time.time() - t1} seconds, which is {len(test_df_semi) / (time.time() - t1)} images per second')

In [ ]:
for i in range(1,10):
    test_df_semi[f'attr_{i}'] = preds[f'attr_{i}']
test_df_semi    

In [ ]:
test_df_semi.to_csv(f'test_validation_df_{test_c_name}.csv',index=False)

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import pickle
from tqdm import tqdm
import time

def load_model(model_path, num_classes_per_attr, device):
    # Initialize the model architecture and load the saved weights
    model = MultiLabelClassifier(num_classes_per_attr)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = torch.nn.DataParallel(model)
    model.to(device)
    model.eval()
    return model
    
def load_label_encoders(encoder_path):
    with open(encoder_path, 'rb') as f:
        encoders = pickle.load(f)
    return encoders
def preprocess_image(image_path, image_size=(512,512)):
    # Define image transformations (same as used during training)
    transform = transforms.Compose([
        transforms.Resize((512,512)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    # Open image and apply transformations
    image = Image.open(image_path).convert('RGB')
    image = transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    return image
    
def inference(image, model, label_encoders):
    # Perform inference
    with torch.no_grad():
        outputs = model(image)
    # Decode predictions
    predicted_labels = []
    for i, output in enumerate(outputs):
        _, predicted = torch.max(output, 1)
        # Attribute name should match the encoder dictionary keys like 'attr_1', 'attr_2', etc.
        attr_name = f'attr_{i + 1}'
        if attr_name in label_encoders.encoders:
            decoded_label = label_encoders.encoders[attr_name].inverse_transform([predicted.item()])[0]
            predicted_labels.append(decoded_label)
        else:
            raise KeyError(f"Encoder for {attr_name} not found in the loaded label encoders.")
    return predicted_labels
# Load the model and encoders once
model_path = f"best_model_end_{test_c_name}.pth"
encoder_path = f"{working_path}/label_encoders_{test_c_name}.pkl"
num_classes_per_attr = NUM_OF_SEMI_CLASSES_OF_COLUMNS
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = load_model(model_path, num_classes_per_attr, device)
label_encoders = load_label_encoders(encoder_path)
image_dir = f"{input_path}/test_images"
preds = {f'attr_{i}': [] for i in range(1, 10)}
t1 = time.time()
for val in tqdm(test_df_semi['id'], desc='Processing Images', total=len(test_df_semi)):
    image_path = f"{image_dir}/{str(val).zfill(6)}.jpg"
    image = preprocess_image(image_path).to(device)  # Preprocess and send image to device
    predictions = inference(image, model, label_encoders)  # Use the already loaded model and encoders
    for i in range(1, 10):
        preds[f'attr_{i}'].append(predictions[i-1])
print(f'Time taken to process images is {time.time() - t1} seconds, which is {len(test_df_semi) / (time.time() - t1)} images per second')

In [ ]:
for i in range(1,10):
    test_df_semi[f'attr_{i}'] = preds[f'attr_{i}']
test_df_semi    

In [ ]:
test_df_semi.to_csv(f'test_df_semi_{test_c_name}.csv',index=False)